In [2]:
import sqlite3
import numpy as np
import pandas as pd
from pandas import HDFStore

import pubchempy as pcp
import json

#from urllib.request import urlopen
import urllib
from urllib.request import urlopen

from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw
from rdkit.Chem import PandasTools


import os as os
from os import system, path, remove
import glob
import time
import shutil

import platform
host = platform.node()

#file modes for chmod commands
mod = 0o755

btime = time.strftime("%Y-%m-%d %H:%M")
#print(btime)



#hostflag = 0
if (host == 'boron' or 'sausage'):
    home = '/home/huffman/work/matinsy/'
    dbfile = home+'db/cheminventory.db'
    webhtmldir = './'
    webmsdsdir = webhtmldir+'msds/'
    websafetyplansdir = webhtmldir+'Lab_Specific_Hygiene_Plans/'
    htmldir = '/home/huffman/public_html/sdsweb/'
    safetyplansdir = htmldir+'Lab_Specific_Hygiene_Plans/'
    safetyplansnoplan = './noplans.html'
    msdsdir = htmldir+'msds/'
    roomfile = home+'etc/allrooms.dat'
    cheminfodir = htmldir+'cheminfo/'
    cheminfodata = home+'cheminfodata/'
    
elif host == 'msds.wcu.edu':
    home = '/wwbintz/'
    dbfile = home+'/matinsy/db/cheminventory.db'
    htmldir = home+'public_html/'
    webhtmldir = './'
    safetyplansdir = htmldir+'Lab_Specific_Hygiene_Plans/'
    safetyplansnoplan = './noplans.html'
    msdsdir = htmldir+'msds/'
    webmsdsdir = webhtmldir+'msds/'
    websafetyplansdir = webhtmldir+'Lab_Specific_Hygiene_Plans/'
    roomfile = home+'matinsy/etc/allrooms.dat'
    cheminfodir = htmldir+'cheminfo/'
    cheminfodata = home+'matinsy/cheminfodata/'
else:
    pass



tp = '<HTML>\n <HEAD><TITLE>SDS chemical inventory DB Problems </TITLE></HEAD>\n<BODY>\n'
hd = '<H1>'  
he = '</H1>\n'
lt = '<UL>'
le = '</UL>'
li = '<LI>'
dn = '</BODY>\n </HTML>\n'

print('********************************************')

bmsg = ' db cheminfo building beginning '
print(host,bmsg,btime)
print(cheminfodata)

********************************************
boron  db cheminfo building beginning  2017-10-13 16:55
/home/huffman/work/matinsy/cheminfodata/


## definitions

In [2]:
def dicwrite(casfile,dic):
    # save to file:
    with open(casfile, 'w') as f:
        json.dump(dic, f)
    
def writehtml(ofile,htmltable,room):
    #room = df.room.unique()
    #room = room.tolist()[0]
    #print(getevaclink(room))
    #print(getchplink(room))
    #print(getsoplinks(room))

    #evaclink = getevaclink(room)
    #chplink = getchplink(room)
    #soplink = getsoplinks(room)
    #table = df.style.applymap(highlight_vals, subset=['regtype']).set_table_attributes("border=1").render()
    #nfpatable = roomdf.to_html(na_rep='0', col_space=12)##ask wes about this
    out = ' '.join(('<HTML>\n <HEAD><TITLE> chemical Information </TITLE> \
        </HEAD>\n<BODY>\n <H1> ',room,
        '<H2> CHEMINFO </H2>',htmltable,\
        '</BODY>\n </HTML>\n'))
    #tp = "<HTML>\n <HEAD><TITLE>SDS chemical inventory  </TITLE> \
    #    </HEAD>\n<BODY>\n                                           \
    #    <H1>Evaculation plans for this room </H1>                    \
    #    <H1> Hygene plans for this room </H1>                      \
    #    <H1> NFPA max scores for this room </H1>                    \
    #    
    #    <H1 style=\"color:red\" > Chemical Inventory  RED are Potentially Hazardous Substances</H1>\n    \
    #    \n"
    
    #dn = '</BODY>\n </HTML>\n'
    with open(ofile, 'w') as f:
        f.write(out)
        #f.write( tp)
        #f.write(df.style.applymap(highlight_vals, subset=['regtype']).set_table_attributes("border=1").render())
        #f.write(dn)
    #os.chmod(ofile, mod)
    return


def getbotsNroom(dbfile,room):
    conn = sqlite3.connect(dbfile)
    c = conn.cursor()
    rooms = []
    catid = []
    CAS = []
    d = {}
    c.execute('select catid,room,CAS,reorder,name from Bot where room=?',[room])
    tmp1 = c.fetchall()
    for i in range(len(tmp1)):
        catid.append(tmp1[i][0])
        rooms.append(tmp1[i][1])
        CAS.append(tmp1[i][2])
        tmp2 = []
        for j in range(1,len(tmp1[i])):
            tmp2.append(tmp1[i][j])
        d[tmp1[i][0]] = tmp2
        conn.commit()
    c.close()
    return d


def getallbots(dbfile):
    conn = sqlite3.connect(dbfile)
    c = conn.cursor()
    rooms = []
    catid = []
    CAS = []
    d = {}
    c.execute('select catid,room,CAS,reorder,name from Bot')
    tmp1 = c.fetchall()
    for i in range(len(tmp1)):
        catid.append(tmp1[i][0])
        rooms.append(tmp1[i][1])
        CAS.append(tmp1[i][2])
        tmp2 = []
        for j in range(1,len(tmp1[i])):
            tmp2.append(tmp1[i][j])
        d[tmp1[i][0]] = tmp2
        conn.commit()
    c.close()
    return d

#cas = '114460-21-8'
#cas = '107-07-3'
#  '0-0-0000',None

def is_number(s):
    try:
        complex(s) # for int, long, float and complex
    except ValueError:
        return False

    return True

def ckcasvalidity(cas):
    #print('CAS',cas)
    exceptions = (';afjadjfakjf;ad','addad22222',None,'na','fake','?','?8-83-4')#('114460-21-8', '1319-46-6',  '9003-70-7', '10486-00-7','9047-08-9','14024-63-6','28053-08-9','58-86-6','8005-03-6','9001-41-6','13520-83-7','13520-83-7','1250-23-4','9000-01-5','12030-88-5','67-09-5')
    if cas in exceptions:
        casflag = 2
        #print('CAS exception')
    elif is_number(cas[0]) == False:
        print(cas)
        casflag = 2
    elif '1-0-' in cas:
        casflag = 3
    else:
        #print(cas)
        cs = cas.split('-')
        #print('cs',cs)
        Nstr = ''.join(cs[:2])
        #print('Nstr',Nstr)
        Q = float(cs[-1])
        #print('Q',Q)
        LN = len(Nstr)
        Nlist = []
        for i,N in enumerate(Nstr):
            Nlist.append(float(N)*(LN-i))
        Nprod = np.mod(np.sum(Nlist),10)
        if Nprod == Q:
            casflag = 1
            #print('Valid CAS')
        else:
            casflag = 0
            #print('Bad CAS',cas,Nprod,Q)
    return casflag
#print(Q,Nprod,Nlist)
def casvalidlist(caslist):
    #TODO:  add wes cas filter
    validlist = []
    invalidlist = []
    exceptlist = []
    wescas = []
    for cas in caslist:
        flag = ckcasvalidity(cas)
        if flag == 1:
            validlist.append(cas)
        elif flag == 0:
            invalidlist.append(cas)
        elif flag == 2:
            exceptlist.append(cas)
        elif flag == 3:
            wescas.append(cas)
        else:
            pass
    return validlist,invalidlist,exceptlist,wescas

In [14]:
outdict = {}

In [15]:
dd = getallbots(dbfile)
ddf = pd.DataFrame.from_dict(dd, orient='index')
ddf.rename(columns={0:'room',1:'CAS',2:'reorder',3:'name'},inplace=True)
caslist = list(filter(None,set(ddf['CAS'].tolist())))
valid,invalid,exclist,wescas = casvalidlist(caslist)

#dfvalid = ddf[ddf.CAS.isin(valid)]
dfinvalid = ddf[ddf.CAS.isin(invalid)]
dfexclist = ddf[ddf.CAS.isin(exclist)]
dfwescas  = ddf[ddf.CAS.isin(wescas)]
#outdict['dfvalid'] = dfvalid
outdict['invalid cas'] = dfinvalid
outdict['completely wrong'] = dfexclist
outdict['wes cas'] = dfwescas
#ddf[ddf.CAS.isin(invalid)]


assign fake


In [16]:
print(len(valid),len(invalid),len(exclist),len(wescas))
print(invalid)
hd = '<H1 style=color:red>\n'  
he = '</H1>\n'
ofile = htmldir+'test.html'
#remove(ofile)
with open(ofile, 'w') as f:
    f.write( tp)
    for key,dfout in outdict.items():
        print(key)
        f.write(hd)
        f.write(key)
        f.write(he)
        f.write(dfout.to_html())
        
    f.write(dn)

2388 91 3 150
['7709-99-8', '64-17-3', '626-5-5-1', '4996-61-3', '13157-24-3', '122-58-3', '87199-17-6', '7882-50-5', '15009-13-5', '50-0-0', '50-01-0', '75-05-1', '657-26-11', '196585-44-9', '7447-40-2', '26718-90-3', '1310-73-5', '8226-79-5', '17398-24-8', '107-89-9', '118-92-0', '13446-34-6', '30813-34-1', '1379-92-7', '0012-80-0', '128-24-3', '137-18-6', '127-08-1', '7987-37-2', '25232-41-4', '72-55-6', '17-1335-01', '9005-54-0', '7704-39-9', '25322-86-3', '7747-40-7', '6199-70-6', '64-18-3', '67-66-1', '15086-94-6', '25839-94-0', '7772-18-7', '7740-43-9', '50-18-7', '9057-37-4', '231-85-3', '1206-17-9', '35952-93-8', '63-03-8', '25332-41-1', '6153-56-3', '13362-70-2', '107-15-1', '630-45-2', '92-70-8', '1250-23-4', '7311-3-0--0', '176-39-2', '211611000', '69-76-3', '131-73-2', '201-45-8', '622-60-3', '60-72-7', '78-38-7', '109-0-4--6', '2883-46-6', '9048-46-86', '88-98-1', '7967-37-2', '626-85-1', '311-39-5', '10376-23-1', '8073-10-8', '28631-60-5', '298-38-9', '24560-42-8', '89-9

In [17]:
#cas = eval('91-01-0')
#print(is_number(cas))
#test  = np.mod(7*6 + 6*6 + 5*4 + 3*6 +2*7 + 8*1,10)
#print(test)
#caslist = list(set(df['CAS'].tolist()))
#valid,invalid,exclist,wescas = casvalidlist(caslist)

In [18]:
cas = '114460-21-8'
cas = '1319-46-6'
cas = '10486-00-7'
cas = None
cas = '521-31-3'
cas = '7646-78-8'
casflag = ckcasvalidity(cas)
casflag

1

In [19]:
room = 'NS202'
d = getbotsNroom(dbfile,room)
df = pd.DataFrame.from_dict(d, orient='index')
df.rename(columns={0:'room',1:'CAS',2:'reorder',3:'name'},inplace=True)

In [20]:
#for cas in invalid:
    #print(df[df['CAS']== cas])

In [21]:
#caslist = df['CAS'].tolist()
#caslist = list(filter(None, set(caslist))) # fastest
notpubchem = []
ciddict = {}
caslist = valid
lcas = len(caslist)
for i,cas in enumerate(caslist):
    print(cas,i,' out of ',lcas)
    casflag = ckcasvalidity(cas)
    if casflag == 0:
        pass
    else:
        url = 'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/'+cas+'/cids/json'
        try:
            response = urllib.request.urlopen(url)
            cid = json.loads(response.read())['IdentifierList']['CID']
            c = pcp.Compound.from_cid(cid)
            smiles = c.isomeric_smiles
            name = c.iupac_name
            mw = c.molecular_weight
            proplist = [cid,name,smiles,mw]
            ciddict[cas] = proplist
            dic = c.to_dict()
            casfile = cheminfodata+cas+'.json'
            dicwrite(casfile,dic)
        except urllib.error.HTTPError as e:
            print(e.code)
            notpubchem.append(cas)
            #print(urllib.error.HTTPError.read())
            pass
        #url = 'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/'+cas+'/cids/json'
        

1945-77-3 0  out of  2388
95-54-5 1  out of  2388
59-88-1 2  out of  2388
92-94-4 3  out of  2388
540-63-6 4  out of  2388
7446-14-2 5  out of  2388
13600-98-1 6  out of  2388
404
108-45-2 7  out of  2388
9005-53-2 8  out of  2388
404
122-99-6 9  out of  2388
1522-22-1 10  out of  2388
2243-62-1 11  out of  2388
2637-34-5 12  out of  2388
38099-82-0 13  out of  2388
404
112-92-5 14  out of  2388
128-80-3 15  out of  2388
26675-46-7 16  out of  2388
144-21-8 17  out of  2388
91-95-2 18  out of  2388
338950-86-0 19  out of  2388
6117-80-2 20  out of  2388
7440-47-3 21  out of  2388
112-14-1 22  out of  2388
7440-42-8 23  out of  2388
140-10-3 24  out of  2388
532-94-5 25  out of  2388
502-49-8 26  out of  2388
67-42-5 27  out of  2388
138495-42-8 28  out of  2388
108-18-9 29  out of  2388
98-95-3 30  out of  2388
9047-08-9 31  out of  2388
404
5794-28-5 32  out of  2388
496-72-0 33  out of  2388
109-94-4 34  out of  2388
24801-88-5 35  out of  2388
61702-42-9 36  out of  2388
306-08-1 37

In [22]:
ciddict

{'1945-77-3': [[73037, 73038, 16212122],
  'tetrasodium;2-[[5-[3-[3-[[bis(carboxylatomethyl)amino]methyl]-4-hydroxy-2-methyl-5-propan-2-ylphenyl]-1,1-dioxo-2,1$l^{6}-benzoxathiol-3-yl]-2-hydroxy-6-methyl-3-propan-2-ylphenyl]methyl-(carboxylatomethyl)amino]acetate',
  'CC1=C(C(=C(C=C1C2(C3=CC=CC=C3S(=O)(=O)O2)C4=CC(=C(C(=C4C)CN(CC(=O)[O-])CC(=O)[O-])O)C(C)C)C(C)C)O)CN(CC(=O)[O-])CC(=O)[O-].[Na+].[Na+].[Na+].[Na+]',
  844.747],
 '95-54-5': [[7243], 'benzene-1,2-diamine', 'C1=CC=C(C(=C1)N)N', 108.144],
 '59-88-1': [[60962],
  'phenylhydrazine;hydrochloride',
  'C1=CC=C(C=C1)NN.Cl',
  144.602],
 '92-94-4': [[7115],
  '1,4-diphenylbenzene',
  'C1=CC=C(C=C1)C2=CC=C(C=C2)C3=CC=CC=C3',
  230.31],
 '540-63-6': [[10902], 'ethane-1,2-dithiol', 'C(CS)S', 94.19],
 '7446-14-2': [[24008],
  'lead(2+);sulfate',
  '[O-]S(=O)(=O)[O-].[Pb+2]',
  303.256],
 '108-45-2': [[7935], 'benzene-1,3-diamine', 'C1=CC(=CC(=C1)N)N', 108.144],
 '122-99-6': [[31236], '2-phenoxyethanol', 'C1=CC=C(C=C1)OCCO', 138.166],
 

In [23]:
dfstruct = pd.DataFrame.from_dict(ciddict,orient='index')
dfstruct.rename(columns={0:'cid',1:'name',2:'smiles',3:'mw'},inplace=True)
PandasTools.AddMoleculeColumnToFrame(dfstruct, smilesCol='smiles', molCol='struct', includeFingerprints=False)
htmltable = dfstruct.to_html()
ofile = cheminfodir+'test'+'.html'
writehtml(ofile,htmltable,room)

RDKit ERROR: [18:02:07] Explicit valence for atom # 1 Bi, 5, is greater than permitted


In [24]:

#valid,invalid,exclist,wescas = casvalidlist(caslist)

In [25]:
store = HDFStore(cheminfodata+'store.h5')



In [26]:
store['df'] = df  # save it
store['dfstruct'] = dfstruct
#store['ddf'] = ddf
#df = store['df']  # load it
store['notdf'] =pd.Series(notpubchem)

/home/huffman/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2881: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->['room', 'CAS', 'reorder', 'name']]

  exec(code_obj, self.user_global_ns, self.user_ns)
/home/huffman/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2881: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->['cid', 'name', 'smiles', 'struct']]

  exec(code_obj, self.user_global_ns, self.user_ns)


In [27]:
nS = store['notdf']
df = store['df']

In [28]:
cas = '9004-67-5'#methyl cellulose
cas = '10049-08-8'#Ruthenium(III) chloride, cid = 61850
cas = '19423-86-0'# Thulium(III) chloride, cid = 61643
cas = '13453-07-1'# gold chloride trihydrate, cid = 44134746
cas = '68855-54-9'# Diatomaceous earth, flux-calcined, 

T = df[df.CAS == cas]
T

,room,CAS,reorder,name


In [29]:
nS

0        13600-98-1
1         9005-53-2
2        38099-82-0
3         9047-08-9
4        10025-77-1
5        39409-82-0
6        12650-88-3
7         8008-20-6
8         9028-48-2
9         9048-46-8
10       15663-27-1
11        9011-14-7
12       26472-00-4
13       13453-07-1
14       12037-29-5
15         109-63-7
16       37222-66-5
17        9007-83-4
18        1314-56-3
19         509-34-2
20        7789-29-9
21       63231-60-7
22         0-0-0000
23        8063-24-9
24        7791-13-1
25        1309-37-1
26      114460-21-8
27        6895-42-7
28        8012-95-1
29       13939-06-5
30        7783-85-9
31        8012-89-3
32        9001-50-7
33        8023-70-9
34        7705-08-0
35       10049-08-8
36       12036-44-1
37        8015-86-9
38        9002-07-7
39        9001-75-6
40          59-23-4
41       12036-41-8
42        2564-83-2
43        9000-70-8
44          58-86-6
45       28631-66-5
46       71010-52-1
47        9001-51-8
48        9004-67-5
49        8013-01-2


In [30]:
notpubchem

['13600-98-1',
 '9005-53-2',
 '38099-82-0',
 '9047-08-9',
 '10025-77-1',
 '39409-82-0',
 '12650-88-3',
 '8008-20-6',
 '9028-48-2',
 '9048-46-8',
 '15663-27-1',
 '9011-14-7',
 '26472-00-4',
 '13453-07-1',
 '12037-29-5',
 '109-63-7',
 '37222-66-5',
 '9007-83-4',
 '1314-56-3',
 '509-34-2',
 '7789-29-9',
 '63231-60-7',
 '0-0-0000',
 '8063-24-9',
 '7791-13-1',
 '1309-37-1',
 '114460-21-8',
 '6895-42-7',
 '8012-95-1',
 '13939-06-5',
 '7783-85-9',
 '8012-89-3',
 '9001-50-7',
 '8023-70-9',
 '7705-08-0',
 '10049-08-8',
 '12036-44-1',
 '8015-86-9',
 '9002-07-7',
 '9001-75-6',
 '59-23-4',
 '12036-41-8',
 '2564-83-2',
 '9000-70-8',
 '58-86-6',
 '28631-66-5',
 '71010-52-1',
 '9001-51-8',
 '9004-67-5',
 '8013-01-2',
 '10476-85-4',
 '1330-20-7',
 '1031265-39-0',
 '37360-75-1',
 '7758-16-9',
 '8049-47-6',
 '37224-29-6',
 '7784-13-6',
 '9003-70-7',
 '12054-85-2',
 '12135-76-1',
 '81028-91-3',
 '20211-76-1',
 '16828-11-8',
 '52785-06-5',
 '199620-18-3',
 '14024-18-1',
 '7772-99-8',
 '34725-61-6',
 '9008

In [31]:
cas = '64-19-7'
url = 'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/'+cas+'/cids/json'
response = urllib.request.urlopen(url)
cid = json.loads(response.read())['IdentifierList']['CID']
c = pcp.Compound.from_cid(cid)
smiles = c.isomeric_smiles
name = c.iupac_name
mw = c.molecular_weight
proplist = [cid,name,smiles,mw]
casfile = cheminfodata+cas+'.json'
cdict = c.to_dict()
with open(casfile, 'w') as f:
    json.dump(cdict, f)
print(casfile)

/home/huffman/work/matinsy/cheminfodata/64-19-7.json


In [32]:
with open(casfile, 'r') as f:
    try:
        data = json.load(f)
    # if the file is empty the ValueError will be thrown
    except ValueError:
        data = {}

# save to file:
with open(casfile, 'w') as f:
    json.dump(cdict, f)

## looking at cameo

In [5]:
casfiles = glob.glob(cheminfodata+'*.json')
casfile = casfiles[0]
with open(casfile, 'r') as f:
    try:
        data = json.load(f)
    # if the file is empty the ValueError will be thrown
    except ValueError:
        data = {}

In [6]:
data


{'atom_stereo_count': 1,
 'atoms': [{'aid': 1, 'element': 'O', 'number': 8, 'x': 7.1962, 'y': 0.134},
  {'aid': 2, 'element': 'O', 'number': 8, 'x': 8.0622, 'y': -1.366},
  {'aid': 3, 'element': 'C', 'number': 6, 'x': 6.3301, 'y': -0.366},
  {'aid': 4, 'element': 'C', 'number': 6, 'x': 5.4641, 'y': -0.866},
  {'aid': 5, 'element': 'C', 'number': 6, 'x': 4.5981, 'y': -0.366},
  {'aid': 6, 'element': 'C', 'number': 6, 'x': 6.8301, 'y': -1.232},
  {'aid': 7, 'element': 'C', 'number': 6, 'x': 5.8301, 'y': 0.5},
  {'aid': 8, 'element': 'C', 'number': 6, 'x': 3.732, 'y': -0.866},
  {'aid': 9, 'element': 'C', 'number': 6, 'x': 2.866, 'y': -0.366},
  {'aid': 10, 'element': 'C', 'number': 6, 'x': 8.0622, 'y': -0.366},
  {'aid': 11, 'element': 'C', 'number': 6, 'x': 6.3301, 'y': 1.366},
  {'aid': 12, 'element': 'C', 'number': 6, 'x': 2, 'y': -0.866},
  {'aid': 13, 'element': 'C', 'number': 6, 'x': 2.866, 'y': 0.634},
  {'aid': 14, 'element': 'C', 'number': 6, 'x': 8.9282, 'y': 0.134},
  {'aid': 